In [1]:
pip install FastKmedoids --upgrade

In [ ]:
from FastKmedoids.models import FastKmedoidsGG

In [ ]:
FastKmedoidsGG

FastKmedoids.models.FastKmedoidsGG

In [ ]:
pip show FastKmedoids

Name: FastKmedoids
Version: 0.0.11
Summary: This is a package to implement a fast and powerful version of the Kmedoids clustering algorithm, which is built on the Generalised Gower distance, already available in the PyDistances package
Home-page: https://github.com/FabioScielzoOrtiz/FastKmedoids_Package
Author: Fabio Scielzo Ortiz
Author-email: fabioscielzo98@gmail.com
License: 
Location: c:\Users\fscielzo\anaconda3\Lib\site-packages
Requires: numpy, polars
Required-by: 
Note: you may need to restart the kernel to use updated packages.
